Word2Vec : 모든 단어를 사용하니까 문제가 생기는 것이니.. 추론 기반의 방법 -> 주변 단어가 주어졌을 때 어떤 단어가 들어갔는지 추측하는 것 

정의 : 단어간 유사도를 반영해 단어를 벡터화 하는 임베딩 방법론. 비슷한 위치의 단어는 비슷한 의미를 가진다는 가정하에 단어 간 유사도를 계산. 

알고리즘 : CBOW, Skip-gram 대부분 skip-gram을 사용 

CBOW : 주변 단어로 중심 단어 예측 
skip-gram : 중심 단어로 주변 단어 예측

입력 값을 One Hot Vector로 만들어서 넣어줌 -> 이를 결합한 은닉층 생성 -> 은닉층과 가중치를 곱해서 Score를 추출 -> Softmax를 통해 확률로 변환 -> Loss를 계산 -> Backpropagation을 통해 가중치를 업데이트 

위 과정을 다른 문맥에서도 반복 수행..! CBOW 는 인풋이 두개, 아웃풋이 하나였으면 skip-gram은 인풋이 하나, 아웃풋이 두개. 

In [ ]:
pip install gensim

In [4]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim 


import warnings
warnings.filterwarnings(action='ignore')

In [5]:
movie = pd.read_csv('study/data/ratings.csv', low_memory=False)
movie.head(2)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179


In [6]:
movie = movie.sort_values(by='timestamp', ascending=True).reset_index(drop=True)
movie.head()

,userId,movieId,rating,timestamp
0,383,21,3.0,789652009
1,383,47,5.0,789652009
2,383,1079,3.0,789652009
3,409,21,5.0,828212412
4,409,25,4.0,828212412


In [7]:
# 영화의 Metadata를 불러와서 movieID에 맞는 TITLE을 구해줍니다. 
meta = pd.read_csv('study/data/movies_metadata.csv', low_memory=False)
meta.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [8]:
meta = meta.rename(columns={'id':'movieId'})
movie['movieId'] = movie['movieId'].astype(str)
meta['movieId'] = meta['movieId'].astype(str)

movie = pd.merge(movie, meta[['movieId', 'original_title']], how='left', on='movieId')

In [9]:
movie = movie[movie['original_title'].notnull()].reset_index(drop=True)


In [10]:
agg = movie.groupby(['userId'])['original_title'].agg({'unique'})
agg.head()


,unique
userId,
1,"[Jay and Silent Bob Strike Back, Vivement dima..."
2,"[Terminator 3: Rise of the Machines, The Conve..."
3,"[300, The Killing, Shortbus, Finding Neverland..."
4,"[David, The Wedding Planner, Casablanca, Sleep..."
5,"[Gleaming the Cube, Cool Hand Luke, Hidalgo, U..."


In [11]:
movie['original_title'].unique()


array(['The Endless Summer', 'Jarhead', '彼女の想いで', ...,
       'The Lonedale Operator', 'Violeta se fue a los cielos',
       'To Kill a Priest'], dtype=object)

In [12]:
sentence = []
for user_sentence in agg['unique'].values:
    sentence.append(list(map(str, user_sentence)))

In [ ]:
sentence

In [17]:
# Word2vec의 학습을 진행해줍니다. 
from gensim.models import Word2Vec
embedding_model = Word2Vec(sentence, vector_size=20, window = 5, 
                           min_count=1, workers=4, epochs=200, sg=1)

In [18]:
embedding_model.wv.most_similar(positive=['Spider-Man 2'], topn=10)


[('Snow Cake', 0.8435748219490051),
 ('Domicile Conjugal', 0.780623197555542),
 ('Face/Off', 0.7798770070075989),
 ('Rumor Has It...', 0.7500918507575989),
 ('Snow White and the Seven Dwarfs', 0.7462812662124634),
 ('Star Trek: Nemesis', 0.7412996888160706),
 ('Fail-Safe', 0.7352067828178406),
 ('Sunrise: A Song of Two Humans', 0.7246354818344116),
 ('Mr. Brooks', 0.7186208367347717),
 ('Heavenly Creatures', 0.7173405885696411)]

컨텐츠 기반 모델의 장점 : 다른 사용자들의 평점 대신 자신의 평점 만으로 추천 시스템을 만들수 있음,
item의 feature를 통해 추천을 해 설명 용이, 
사용자가 평점을 매기지 않은 새로운 item에도 추천 용이 

단점 : 제대로 추출하지 못하면 정확도가 낮음 (도메인 지식이 필요), meta 데이터 활용이 힘듬

따라서 협업 필터링 모델과의 콜라보레이션이 필요함! 